<a href="https://colab.research.google.com/github/nurimammasri/Wooky-Machine-Learning/blob/main/Final%20Project/Final%20Project%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# File Notebook untuk Machine Learning Competition

In [1]:
import numpy as np
import pandas as pd

#load data csv
df1 = pd.read_csv('https://drive.google.com/uc?id=14yUYJH-tz-tj4oOiGn7Bkbjh0cz-Y0OX')
df2 = pd.read_csv('https://drive.google.com/uc?id=1-E8qd6nEIqHK774rUVaWhEDBRuwA113w')

df1=pd.read_csv('https://raw.githubusercontent.com/nurimammasri/Wooky-Machine-Learning/main/Data/final%20project/train.csv')
df2=pd.read_csv('https://raw.githubusercontent.com/nurimammasri/Wooky-Machine-Learning/main/Data/final%20project/test.csv')

## Tahap pertama adalah memanipulasi data agar ia dapat diproses pada model machine learning selanjutnya

In [2]:
#Sebelum diload, saya sudah periksa data csv, dan ia mengandung data kosong dengan value string '?'
#Replace data kosong dengan NaN
df1 = df1.replace('?', np.nan)
df2 = df2.replace('?', np.nan)

In [3]:
#Isi data NaN dengan method frontfill
df1 = df1.fillna(method='ffill')
df2 = df2.fillna(method='ffill')

In [4]:
#Encode data untuk kolom berisi data string
df1 = pd.get_dummies(df1, columns=['Kelas Pekerja', 'Status Perkawinan',
             'Pekerjaan'])
df2 = pd.get_dummies(df2, columns=['Kelas Pekerja', 'Status Perkawinan',
             'Pekerjaan'])

In [5]:
#Replace kolom pendidikan dengan data ordeal
col1 = {'SD':1,'1st-4th':2, '5th-6th':3, '7th-8th':4, 
        '9th':5, '10th':6, '11th':7, '12th':8, 'SMA':9, 
        'Pendidikan Tinggi':10, 'D4':11, 'D3':12, 'Sarjana':13, 
        'Master':14, 'Sekolah Professional':15, 'Doktor':16 }
df1['Pendidikan'] = df1['Pendidikan'].replace(col1)
df2['Pendidikan'] = df2['Pendidikan'].replace(col1)

In [6]:
#replace kolom jenis kelamin dengan data ordeal
col2 = {'Laki2':1, 'Perempuan':2}
df1['Jenis Kelamin'] = df1['Jenis Kelamin'].replace(col2)
df2['Jenis Kelamin'] = df2['Jenis Kelamin'].replace(col2)

In [7]:
#replace data label
col3 = {'<=7jt':0, '>7jt':1}
df1['Gaji'] = df1['Gaji'].replace(col3)

## Untuk percobaan model, saya akan menggunakan dua model, yaitu KNN dan Decision Tree Regressor

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [9]:
#Pemilihan data train, dan data test
X_train = df1.drop('Gaji', axis=1)
y_train = df1['Gaji']
y_train = y_train

X_test = df2

### Model Pertama: KNN

In [10]:
#Percobaan pertama menggunakan model KNN
#Ujicoba tuning hyperparameter KNN
model = KNeighborsClassifier()
param_grids = {
    'n_neighbors':np.arange(10,40), 
    'weights': ['uniform','distance'], 
}
rscv = RandomizedSearchCV(model, param_distributions=param_grids, n_iter=25, scoring='roc_auc', cv=5)
rscv.fit(X_train,y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='deprecated', n_iter=25, n_jobs=None,
                   param_distributions={'n_neighbors': array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]),
                                        'weights': ['uniform', 'distance']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='roc_auc', verbose=0)

In [11]:
#dapatkan hyperparameter terbaik
rscv.best_params_

{'n_neighbors': 24, 'weights': 'distance'}

In [12]:
#dapatkan score terbaik
rscv.best_score_

0.6758224149284605

In [13]:
#Gunakan hyperparameter terbaik setelah tuning pada model KNN
model = KNeighborsClassifier(n_neighbors=24, weights='distance')
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=24, p=2,
                     weights='distance')

In [14]:
#prediksi data test model KNN
y_pred = model.predict(X_test)

In [15]:
#Ubah data test menjadi data Frame
cek = pd.DataFrame(y_pred, columns=['Gaji'])

In [16]:
#Simpan data test pada file csv
#Input index di sini masih dimulai dari nol
#Untuk urutan index dimulai dari 35994 diinput manual pada saat membuka file csv
cek.to_csv('prediction1.csv')

### Model Kedua: Decision Tree Regressor

In [17]:
from sklearn.tree import DecisionTreeRegressor

In [18]:
#Percobaan kedua menggunakan model Decision Tree Regressor
#Ujicoba tuning hyperparameter Decision Tree Regressor
#Untuk menghemat waktu, saya lakukan per-penilaian per-parameter. Selalu didapat untuk max depth nilai 8 adalah terbaik
#Begitu pula nilai hyperparameter selanjutnya
model2 = DecisionTreeRegressor()
param2_grids = {
    'max_depth':np.arange(1,10),
    'min_samples_split':[5],
    'min_samples_leaf':np.arange(1,5)
}
gscv = GridSearchCV(model2, param_grid=param2_grids, scoring='roc_auc', cv=10)
gscv.fit(X_train,y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_leaf': array([1, 2, 3, 4]),
              

In [19]:
#dapatkan hyperparameter terbaik
gscv.best_params_

{'max_depth': 8, 'min_samples_leaf': 3, 'min_samples_split': 5}

In [20]:
#dapatkan score terbaik
gscv.best_score_

0.90291848701466

In [21]:
#Gunakan hyperparameter terbaik setelah tuning pada model Decision Tree Regressor
model2 = DecisionTreeRegressor(max_depth=8, min_samples_leaf=3, min_samples_split=5)
model2.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=8,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=5,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [22]:
#prediksi data test model Decision Tree Regressor
y_pred = model2.predict(X_test)

In [23]:
#Ubah data test menjadi data Frame
cek = pd.DataFrame(y_pred, columns=['Gaji'])

In [24]:
#Simpan data test pada file csv
#Input index di sini masih dimulai dari nol
#Untuk urutan index dimulai dari 35994 diinput manual pada saat membuka file csv
cek.to_csv('prediction2.csv')

## Sejauh ini pada percobaan saya, model Decision Tree Regressor menghasilkan scoring lebih besar dibandingkan dengan model KNN. Dapat dikatakan bahwa, model Decision Tree Regressor adalah model yang cocok untuk data ini.